### Test lại sau khi cải tiến

In [1]:
import requests
import json
import time
from datetime import datetime
from requests.exceptions import ConnectionError, Timeout, RequestException

# --- CẤU HÌNH ---
API_URL = "http://127.0.0.1:8000/chat"
INPUT_FILE = "full_rag_test_suite_V2.json"
OUTPUT_FILE = f"test_again_V2.json"

# --- CẤU HÌNH TỐC ĐỘ & KIÊN TRÌ ---
REQUEST_TIMEOUT = 120   # Chờ server xử lý tối đa 2 phút
MAX_RETRIES = 5         # Số lần thử lại nếu lỗi kết nối
SAFE_DELAY = 6          # [QUAN TRỌNG] Nghỉ cứng 6s sau mỗi câu thành công 
                        # (Đảm bảo tối đa 10 request/phút -> Rất an toàn cho Gemini Flash)

def process_single_question(item):
    """
    Gửi câu hỏi và xử lý retry thông minh:
    - Nếu lỗi kết nối: Retry sau 10s.
    - Nếu lỗi quá tải (429): Retry sau 30-60s.
    - Chỉ dừng khi thành công hoặc hết số lần thử.
    """
    question = item["input_text"]
    image_input = item.get("input_image")
    payload = {"question": question}
    if image_input:
        payload["image_context"] = image_input

    retries = 0
    
    while retries < MAX_RETRIES:
        try:
            start_time = time.time()
            # Gửi request (Blocking)
            response = requests.post(API_URL, json=payload, timeout=REQUEST_TIMEOUT)
            latency = round(time.time() - start_time, 2)

            # --- PHÂN TÍCH STATUS CODE ---
            
            # 1. Thành công (200) -> Trả về ngay
            if response.status_code == 200:
                return response, latency, None
            
            # 2. Bị quá tải (429 Too Many Requests) hoặc Server quá bận (503)
            # Đây là lúc Gemini kêu cứu, cần nghỉ lâu
            elif response.status_code in [429, 503]:
                wait_time = 30 + (retries * 10) # Tăng dần: 30s, 40s, 50s...
                print(f"      ⚠️ API Quá tải (HTTP {response.status_code}). Đang làm mát {wait_time}s...")
                time.sleep(wait_time)
                retries += 1
                continue # Thử lại câu này

            # 3. Các lỗi logic khác (400, 500 internal code error)
            # Vẫn trả về để ghi log, không retry (vì retry khả năng cao vẫn lỗi code)
            else:
                return response, latency, f"HTTP_FAIL_{response.status_code}"

        except (ConnectionError, ConnectionRefusedError):
            print(f"      ⚠️ Mất kết nối tới Server. Đợi 10s kết nối lại...")
            time.sleep(10)
            retries += 1
            
        except Timeout:
            latency = round(time.time() - start_time, 2)
            print(f"      ⚠️ Timeout (Server treo > {REQUEST_TIMEOUT}s). Bỏ qua.")
            return None, latency, "TIMEOUT_120S"

        except Exception as e:
            print(f"      ⚠️ Lỗi lạ: {str(e)}")
            return None, 0, str(e)

    return None, 0, "SERVER_DOWN_OR_MAX_RETRIES"

def run_tests_safe_mode():
    print(f"🚀 Bắt đầu chế độ AN TOÀN (Gemini Flash Friendly)")
    print(f"⏳ Cấu hình: Timeout {REQUEST_TIMEOUT}s | Safe Delay {SAFE_DELAY}s")

    try:
        with open(INPUT_FILE, "r", encoding="utf-8") as f:
            dataset = json.load(f)
    except FileNotFoundError:
        print("❌ Không tìm thấy file input.")
        return

    results = []
    
    print(f"{'ID':<4} | {'Category':<15} | {'Latency':<8} | {'Status'}")
    print("-" * 65)

    for idx, item in enumerate(dataset):
        q_id = idx + 1
        category_label = f"{item['category'][:3]}-{item['sub_category']}" 
        
        # --- BƯỚC 1: XỬ LÝ ---
        response, latency, error_msg = process_single_question(item)
        
        # --- BƯỚC 2: GHI NHẬN KẾT QUẢ ---
        final_status = "UNKNOWN"
        actual_res = ""
        
        if error_msg:
            if "HTTP_FAIL" in error_msg and response:
                final_status = f"FAIL ({response.status_code})"
                actual_res = response.text
            else:
                final_status = error_msg
                actual_res = f"Error: {error_msg}"
        elif response:
            final_status = "DONE (200)"
            try:
                data = response.json()
                actual_res = data.get("answer", str(data))
            except:
                actual_res = response.text

        # In kết quả
        cat_print = (category_label[:13] + '..') if len(category_label) > 13 else category_label
        print(f"{q_id:<4} | {cat_print:<15} | {latency:<6}s  | {final_status}")

        results.append({
            "id": item["id"],
            "input": item["input_text"],
            "latency": latency,
            "status": final_status,
            "response": actual_res
        })

        # --- BƯỚC 3: NGHỈ GIỮA HIỆP (QUAN TRỌNG) ---
        # Bất kể câu trả lời nhanh hay chậm, luôn nghỉ cứng SAFE_DELAY giây
        # Để đảm bảo RPM (Request Per Minute) không vượt quá ngưỡng
        time.sleep(SAFE_DELAY) 

    # Lưu file
    final_output = {"details": results}
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(final_output, f, ensure_ascii=False, indent=2)
    print(f"\n✅ Hoàn tất. Check file: {OUTPUT_FILE}")

if __name__ == "__main__":
    run_tests_safe_mode()

🚀 Bắt đầu chế độ AN TOÀN (Gemini Flash Friendly)
⏳ Cấu hình: Timeout 120s | Safe Delay 6s
ID   | Category        | Latency  | Status
-----------------------------------------------------------------
1    | IN_-Transport.. | 1.48  s  | DONE (200)
2    | OUT-General_O.. | 4.94  s  | DONE (200)
3    | TEC-Security    | 2.66  s  | DONE (200)
4    | TEC-Security    | 3.89  s  | DONE (200)
5    | TEC-Security    | 3.13  s  | DONE (200)
6    | OUT-General_O.. | 3.35  s  | DONE (200)
7    | TEC-Security    | 2.8   s  | DONE (200)
8    | OUT-Transport.. | 1.85  s  | DONE (200)
9    | OUT-General_O.. | 3.33  s  | DONE (200)
10   | OUT-General_O.. | 2.79  s  | DONE (200)
11   | OUT-General_O.. | 3.27  s  | DONE (200)
12   | IN_-Culture_T.. | 2.38  s  | DONE (200)
13   | OUT-General_O.. | 3.9   s  | DONE (200)
14   | IN_-Culture_T.. | 3.9   s  | DONE (200)
15   | IN_-Transport.. | 3.02  s  | DONE (200)
16   | IN_-Transport.. | 0.32  s  | DONE (200)
17   | IN_-Transport.. | 0.79  s  | DONE (200)
18